In [1]:
import numpy as np
import pandas as pd

df_eddies = pd.read_pickle('/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/df_eddies_processed_1462_10650.pkl')
sample_eddies = [474, 971, 1917, 1978, 33, 207, 1783, 2370]
df_sample = df_eddies[df_eddies.Eddy.isin(sample_eddies)].copy()


In [3]:
import netCDF4 as nc
import numpy as np
from scipy.interpolate import interp1d

# fnumbers = [f'{num:05}' for num in range(1461, 3261+1, 30)] # last valid file 10641
fnumbers = [int(fname[-8:-3]) for fname in df_sample.fname.unique()]
fnumbers = np.array(fnumbers)[np.array(fnumbers) > 3261]

z_r = np.load('/srv/scratch/z5297792/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))

for fnumber in fnumbers:
    print(fnumber)
    fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_{fnumber:05}.nc'
    dataset = nc.Dataset(fname)

    rho = dataset.variables['rho']
    rho = np.transpose(rho, axes=(3, 2, 1, 0))
    rho = np.flip(rho, axis=2)
    depths = z_r[150,150,:]

    def depth_interpolator(M,z_r,depths):

        # Dimensions of M and z_r
        dim_i, dim_j, dim_sigma, dim_time = M.shape
        assert z_r.shape == (dim_i, dim_j, dim_sigma)

        # Initialize interpolated matrix
        M_interpolated = np.zeros((dim_i, dim_j, len(depths), dim_time))

        # Interpolate for each day (time dimension)
        for t in range(dim_time):
            for i in range(dim_i):
                for j in range(dim_j):
                    # Get depth levels from z_r
                    depth_levels = z_r[i, j, :]
                    
                    # Get values of M at each depth level for current time t
                    values = M[i, j, :, t]
                    
                    # Perform interpolation
                    if np.all(np.isnan(values)):
                        # Handle case where all values are NaN (e.g., outside valid data range)
                        interpolated_values = np.full(len(depths), np.nan)
                    else:
                        # Use linear interpolation
                        interpolator = interp1d(depth_levels, values, kind='linear', bounds_error=False, fill_value=np.nan)
                        interpolated_values = interpolator(depths)
                    
                    # Assign interpolated values to M_interpolated for current time t
                    M_interpolated[i, j, :, t] = interpolated_values
            M_interpolated[:,:, 0, t] = M[:,:,0,t]

        return M_interpolated

    rho = depth_interpolator(rho, z_r, depths)

    np.save(f'/srv/scratch/z5297792/Climatology/rhos/rho_{fnumber:05}.npy', rho)
    
    del rho

4461
4491
4521
4551
4581
4611
4641
4671
4701
6981
7011
7041
7071
7101
7131
7431
7461
7491
7671
7701
7731
7761
7791
7821
8901
8931
8961
